In [1]:
import re, cv2, os
import pickle
import numpy as np
from tqdm import tqdm
from PIL import Image
import face_recognition
from sklearn.model_selection import train_test_split

import tensorflow as tf

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.compat.v1.Session(config=config)

from tensorflow.keras import backend as K
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Input, Lambda, Dense, Dropout, LSTM
from keras.layers import Subtract, Convolution2D, MaxPooling2D, Flatten, GlobalMaxPooling2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.vgg16 import VGG16
from efficientnet.tfkeras import EfficientNetB3

In [ ]:
path = 'dataset/train/'
data = []
labels = []
for folders in tqdm(os.listdir(path)):
    folder = path + folders
    for files in os.listdir(folder):
        img = folder + '/' + files
        image = cv2.imread(img)                
        image = cv2.resize(image,(64,64))
        data.append(image)
        labels.append(int(folders))

In [ ]:
data = np.array(data)
labels = np.array(labels)
print(data.shape,labels.shape)

In [ ]:
def make_pairs(data, labels):
    pairData = []
    pairLabels = []
    numClasses = len(np.unique(labels))
    idx = [np.where(labels == i)[0] for i in range(0, numClasses)]
    for idxA in range(len(data)):
        currentAudio = data[idxA]
        label = labels[idxA]
        idxB = np.random.choice(idx[label])
        posData = data[idxB]
        pairData.append([currentAudio, posData])
        pairLabels.append([1])
        negIdx = np.where(labels != label)[0]
        negData = data[np.random.choice(negIdx)]
        pairData.append([currentAudio, negData])
        pairLabels.append([0])
    return (np.array(pairData), np.array(pairLabels))

In [ ]:
%%time
X, Y = make_pairs(data, labels)
print(X.shape,Y.shape)

In [ ]:
input_dim = X.shape[2:]
print(input_dim)

In [ ]:
def siamese_model(input_dim):
    #base_model = EfficientNetB3(weights='imagenet', include_top=True, input_shape=(300, 300, 3))
    #base_model = ResNet50(weights='imagenet', include_top=True, input_shape=input_dim)
    base_model = VGG16(include_top=False, input_shape=input_dim)
    for layers in base_model.layers:
        layers.trainable = False
    base_model = Model(inputs = base_model.input, outputs = base_model.layers[-2].output)
    base_model.summary()
    return base_model

feature_model = siamese_model(input_dim)

In [ ]:
input_shape = input_dim
img_a_in = Input(shape = input_shape, name = 'ImageA_Input')
img_b_in = Input(shape = input_shape, name = 'ImageB_Input')

img_a_feat = feature_model(img_a_in)
img_b_feat = feature_model(img_b_in)

In [ ]:
img_a_feat.shape

In [ ]:
def euclidean_distance(vectors):
    (featsA, featsB) = vectors
    sumSquared = K.sum(K.square(featsA - featsB), axis=1,keepdims=True)
    return K.sqrt(K.maximum(sumSquared, K.epsilon()))

In [ ]:
from tensorflow.keras.layers import concatenate,BatchNormalization,Activation  

combined_features = concatenate([img_a_feat, img_b_feat], name = 'merge_features')
combined_features = Dense(256, activation = 'relu')(combined_features)
combined_features = BatchNormalization()(combined_features)
combined_features = Dropout(0.2)(combined_features)
combined_features = Dense(128, activation = 'relu')(combined_features)
combined_features = BatchNormalization()(combined_features)
distance = Activation('relu')(combined_features)

#distance = Lambda(euclidean_distance)([img_a_feat, img_b_feat])

combined_features = Dense(1, activation = 'sigmoid')(distance)
similarity_model = Model(inputs = [img_a_in, img_b_in], outputs = [combined_features], name = 'Similarity_Model')
similarity_model.summary()

In [ ]:
similarity_model.compile(loss='binary_crossentropy', optimizer="rmsprop", metrics=['accuracy'])

In [ ]:
img_1 = X[:, 0]
img2 = X[:, 1]

In [ ]:
similarity_model.fit([img_1, img2], Y[:],batch_size=1, validation_split=0.2, verbose=1, epochs=1000)

In [ ]:
similarity_model.save("face_siam.h5")

In [ ]:
from tensorflow.keras.models import load_model


similarity_model = load_model("face_siam.h5", custom_objects={'loss': euclidean_distance})

In [ ]:
img1 = cv2.imread("test/1.jpg")
rgb = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB) 
face_locations = face_recognition.face_locations(rgb)
for face_location in face_locations:  
    top, right, bottom, left = face_location
    face1 = img1[top:bottom, left:right] 
    cv2.imwrite("face1.jpg", face1)
    face1 = cv2.resize(face1,(224,224))  
    face1 = np.expand_dims(face1,axis=0)

img2 = cv2.imread("test/1.jpg")
rgb2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB) 
face_locations = face_recognition.face_locations(rgb2)
for face_location in face_locations:  
    top, right, bottom, left = face_location
    face2 = img2[top:bottom, left:right] 
    cv2.imwrite("face2.jpg", face2)
    face2 = cv2.resize(face2,(224,224))
    face2 = np.expand_dims(face2,axis=0)   

pred = similarity_model.predict([face1, face2])[0]
print(pred)

In [ ]:
import os

os.system("shutdown /s /t 1")